In [7]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import wget

In [2]:
dlib.__version__

'19.19.0'

In [ ]:
#This will download the files necessary for this code to run.
#Afterdownlaoding the file extract them.
!python -m wget "http://dlib.net/files/shape_predictor_5_face_landmarks.dat.bz2"
!python -m wget "http://dlib.net/files/dlib_face_recognition_resnet_model_v1.dat.bz2"

In [3]:
#Decompress the above file
detector = dlib.get_frontal_face_detector()
sp = dlib.shape_predictor('shape_predictor_5_face_landmarks.dat')
fr = dlib.face_recognition_model_v1('dlib_face_recognition_resnet_model_v1.dat')

In [4]:
threshold = 0.6
def euclideandistance(source_represtentation, test_representation, img):
    val = img.split('.')[0]
    dis = source_represtentation - test_representation
    dis = np.sum(np.multiply(dis, dis))
    dis = np.sqrt(dis)
    if dis < threshold:
        return val
    else:
        return f'Not {val}'

In [5]:
#Give the image file with your name 
image = 'user_name.jpg'
img = cv2.imread(image)
res = cv2.resize(img, (480, 480))
faces = detector(res, 1)
img1_shape =sp(res, faces[0])
img_aligned = dlib.get_face_chip(res, img1_shape)
img_rep = fr.compute_face_descriptor(img_aligned)
image1 = np.array(img_rep)

In [6]:
#This code will detect if the person in front of camera is the same person as in image or not.
#works for only one person at a time
cam = cv2.VideoCapture(0)

if not cam.isOpened():
    cam = cv2.VideoCapture(0)
    if not cam.isOpened():
        cam = cv2.VideoCapture(0)
else:
    while True:
        _, frame = cam.read()
        img_detected = detector(frame, 1)
        im = list(img_detected)
        #print(len(im))
        if len(im) < 1:
            cv2.imshow('frame',frame)
        else:
            img_shape = sp(frame, img_detected[0])
            img_aligned = dlib.get_face_chip(frame, img_shape)
            img_rep = fr.compute_face_descriptor(img_aligned)
            image2 = np.array(img_rep)
            distance = euclideandistance(image1, image2, image)
            for face in img_detected:
                x = face.left()
                y = face.top() #could be face.bottom() - not sure
                w = face.right() - face.left()
                h = face.bottom() - face.top()
                cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 255), 2)
                cv2.putText(frame, f'{distance}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0 ,255), 2)
            #print(distance, x, y, w, h)
            #cv2.putText
        cv2.imshow('frame',frame)
        if cv2.waitKey(2) & 0xFF ==ord('q'):
            break
cam.release()
cv2.destroyAllWindows()